In [1]:
!pip install python-dotenv
!pip install openai

# Getting data from Mergeflow's API

In [2]:
import requests
import json
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file
mergeflow_api_key = os.getenv('MERGEFLOW_API_KEY')

# the API call
query = 'Factory+Automation' # topic is "large language model"
dataset = '&q=*[MarketSize]' # dataset: Tech blogs
rows = str(3) # first 3 docs only

# in the url below, replace "fw" by your user name. You can see your user name at mergeflow.net
# -> Account -> the string in angle brackets
url = 'https://mergeflow.net/api/v1/lutzenrath/search/getContent?q=' + query + dataset + '&rows=' + rows

# Define the headers to be sent with the request
headers = {
    'MergeflowNet-Api-Auth-Key': mergeflow_api_key
}

# Get and print the response
response = requests.post(url, headers=headers)

mergeflow_docs = []

# Check the response status code and content
if response.status_code == 200:
    mergeflow_response_json = response.json()
    
    # Pretty-print the JSON response -- useful to understand what the JSON looks like
    #pretty_json = json.dumps(mergeflow_response_json, indent=4)
    #print(pretty_json)
    
    # get title, URL, date, and content
    for document in mergeflow_response_json['Documents']:
        current_doc = {
            'title': document['Title'],
            'date': document['Date'],
            'url': document['Url'],
            'content': document['Content'][:5000]
        }
        
        mergeflow_docs.append(current_doc)
        #print(current_doc['content'])        

    print('Found ' + str(len(mergeflow_docs)) + ' documents.')

else:
    print('Request failed with status code:', response.status_code)
    print(response.text)  # Print the response text


Found 3 documents.


# Answering questions on the results with OpenAI GPT

In [4]:
import openai

openai.api_key = os.getenv('OPENAI_API_KEY')

system_prompt = f"""
First, please take a break.
After taking your break, you become a partner at a leading venture capital firm with years of experience in assessing start-ups and business ideas.
You will consider the idea given below and rank where in the innovation curve this technology currently is.

An innovation curve, often referred to as a technology adoption lifecycle or diffusion of innovations curve, is a graphical representation that illustrates the adoption or acceptance of a new product, technology, or innovation over time among members of a given market or society. This curve is typically depicted as a bell-shaped curve when plotted on a graph, showing how the innovation spreads through different segments of adopters.

The innovation curve is usually divided into several categories representing the sequence in which different groups adopt new technology:

Innovators (2.5%): These are the first individuals to adopt an innovation. They are willing to take risks, youngest in age, have the highest social class, have great financial liquidity, are very social and have closest contact to scientific sources and interaction with other innovators. Their risk tolerance allows them to adopt technologies that may ultimately fail. Financial resources help absorb these failures.
Early Adopters (13.5%): This group is next to adopt an innovation. They have the highest degree of opinion leadership among the other adopter categories. Early adopters are typically younger in age, have higher social status, more financial liquidity, advanced education, and are more socially forward than late adopters. They are more discrete in adoption choices than innovators and realize judicious choice of adoption will help them maintain a central communication position.
Early Majority (34%): These individuals adopt an innovation after a varying degree of time. This period is significantly longer than the innovators and early adopters. Early Majority tend to be slower in the adoption process, have above average social status, contact with early adopters, and seldom hold positions of opinion leadership in a system.
Late Majority (34%): Individuals in this category will adopt an innovation after the average member of the society. These individuals approach an innovation with a high degree of skepticism and after the majority of society has adopted the innovation. Late Majority are typically skeptical about an innovation, have below average social status, very little financial liquidity, in contact with others in late majority and early majority, very little opinion leadership.
Laggards (16%): This group is the last to adopt an innovation. Unlike some of the previous categories, individuals in this category show little to no opinion leadership. These individuals typically have an aversion to change-agents and tend to be advanced in age. Laggards typically tend to be focused on “traditions”, likely to have lowest social status, lowest financial liquidity, be oldest of all other adopters, in contact with only family and close friends, very little to no opinion leadership.
Understanding the innovation curve is crucial for businesses and marketers as it helps in developing strategies for product launches and marketing campaigns, targeting each group appropriately to maximize adoption and market penetration.

The fate of the world depends on the quality of your assessment, be extremely diligent.


        """


user_prompt = f"""

Please consider the following idea, delimited by ```

```
Smart Modular Automation Systems with AI-driven Predictive Maintenance and Augmented Reality Assistance
Concept Overview:

The core of this innovation is to develop a factory automation system that is modular, AI-enhanced, and augmented reality (AR) supported. This system would not only automate routine tasks but also offer predictive maintenance capabilities and AR-based assistance for complex manual interventions. The idea combines IoT (Internet of Things) sensors, machine learning algorithms, and AR technologies to create a highly adaptive and intelligent manufacturing environment.

Key Components:

Modular Automation Units: Design the production line with modular units that can be easily reconfigured or scaled according to production needs. Each module can perform different tasks, and when combined, they offer a complete manufacturing solution that can be customized for various products.
AI-driven Predictive Maintenance: Integrate machine learning algorithms that analyze data from IoT sensors embedded in machinery to predict failures before they occur. This predictive maintenance system ensures high uptime for manufacturing equipment by scheduling maintenance tasks just in time to prevent unexpected breakdowns.
Augmented Reality Assistance: Equip maintenance staff and operators with AR glasses that provide real-time data overlay and step-by-step guidance during maintenance or complex machine setups. This could include highlighting parts that need attention, displaying instructions for repair, or even offering virtual assistance from remote experts.
Innovative Features:

Flexibility and Scalability: Easily adapt and scale the production line for different products without significant downtime or reconfiguration efforts.
Reduced Downtime: Significantly lower the risk of unexpected machinery failures and reduce downtime with AI-driven predictive maintenance.
Efficiency in Operations: Enhance operational efficiency by providing workers with AR-based assistance, reducing the time and errors in manual tasks.
Energy and Cost Savings: Optimize energy use and operational costs by using data analytics to streamline processes and maintenance schedules.
Remote Expert Assistance: Leverage AR for remote expert assistance, allowing for immediate problem-solving without the need for experts to be physically present.
Implementation Considerations:

Invest in R&D to develop or adapt modular units for various manufacturing tasks.
Partner with technology providers specializing in AI, IoT, and AR to integrate these technologies into the factory environment.
Train staff to work with new systems and technologies, emphasizing flexibility and adaptability.
This innovative approach not only enhances the efficiency and flexibility of manufacturing operations but also paves the way for future advancements in factory automation by creating a foundation that is adaptable, intelligent, and highly integrated.
```

Please rank this idea in the innovation curve and give reasons why you rank it the way you do.
    
"""
    
completion = openai.chat.completions.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}
    ],
    temperature=0.7,
    max_tokens=2000,
    top_p=0.95,
    frequency_penalty=0,
    presence_penalty=0,
    stop=None
)

print(completion.choices[0].message)  
print(completion.choices[0].message.content) # This prints the content of the output in a more readable way
print("\n---\n")  # This adds a separator between the content blocks for readability 

ChatCompletionMessage(content='Based on the provided description, the idea of Smart Modular Automation Systems with AI-driven Predictive Maintenance and Augmented Reality Assistance can be ranked as follows:\n\nEarly Adopters (13.5%): This technology is likely in the early adopters stage of the innovation curve. It involves the integration of multiple advanced technologies such as IoT, machine learning, and augmented reality into a manufacturing environment. Early adopters are typically open to embracing new technologies and have the resources and willingness to invest in innovative solutions.\n\nReasons for ranking:\n1. Integration of advanced technologies: The idea combines IoT sensors, machine learning algorithms, and augmented reality to create a highly adaptive and intelligent manufacturing environment. This integration is likely to attract early adopters who are interested in exploring and implementing cutting-edge technologies.\n\n2. Potential for significant benefits: The innov